### Varimax Rotation

In [ ]:
import os
import sys
import numpy as np
import scipy as sp
import joblib as jl
import dill as pickle

from tqdm import tqdm
from typing import *

In [ ]:
import matplotlib.pyplot as plt
style = "/home/zanardi/Workspace/Research/styles/matplotlib/paper_1column.mplstyle"
plt.style.use(style)

In [ ]:
sys.path.append("/home/zanardi/Codes/ML/ROMAr/romar/")
from romar import env
from romar import utils
from romar.systems import BoxAd

Set enviroment

In [ ]:
env_opts = {
  "backend": "numpy",
  "device": "cpu",
  "device_idx": 0,
  "nb_threads": 2,
  "epsilon": None,
  "floatx": "float64",
  "seed": 0
}
env.set(**env_opts)

Set inputs

In [ ]:
# Trajectories indices
irange = [0, 100]
# Parallel workers
nb_workers = 10
# Paths
prefix = "/home/zanardi/Codes/ML/ROMAr/runs/run01/"
paths = {
  # > ROM basis
  "roms": {
    "No Rot.": prefix + "/max_mom_2/models/cobras/basis.p",
    "Varimax": prefix + "/max_mom_2/models/cobras/basis_varimax.p"
  },
  # > Path to solutions folder
  "data": prefix + "/data/test/",
  # > Thermochemical database
  "dtb": "/home/zanardi/Codes/ML/ROMAr/romar/examples/database/",
  # > Output folder
  "out": "./figs/varimax/"
}
# Time limit
tlim = [1e-9, 1e-3]
# Number of ROM dimensions
rdims = np.arange(7,10,2)

In [ ]:
os.makedirs(paths["out"], exist_ok=True)

Initialize 0D thermochemical system

In [ ]:
system = BoxAd(
  species={k: paths["dtb"] + "/species/" + k + ".json" for k in ("Ar", "Arp", "em")},
  kin_dtb=paths["dtb"] + "/rates/kin_fit.p",
  rad_dtb=paths["dtb"] + "/rates/rad_fit.p",
  use_rad=True,
  use_proj=False,
  use_tables=False
)

Evaluate ROM

In [ ]:
def evaluate_parallel(
  irange,
  nb_workers,
  **kwargs
):
  iterable = tqdm(
    iterable=range(*irange),
    ncols=80,
    desc="  Cases",
    file=sys.stdout
  )
  return jl.Parallel(nb_workers)(
    jl.delayed(
      env.make_fun_parallel(evaluate)
    )(index=i, **kwargs) for i in iterable
  )

def evaluate(
  system,
  path,
  index,
  tout=5e2,
  tlim=None
):
  system.use_rom = True
  # Load test case
  icase = utils.load_case(path=path, index=index)
  t, y0, rho = [icase[k] for k in ("t", "y0", "rho")]
  # Time window
  if (tlim is not None):
    i = (t >= np.amin(tlim)) * (t <= np.amax(tlim))
    t = t[i]
  # Solve ROM
  z, _ = system.solve_rom(t, y0, rho, tout=tout, decode=False)
  # Postprocess
  if (z is not None):
    # > Return data
    return np.abs(z[:system.rom.size_zhat]).T

def build_rom(system, path_to_basis, rdim):
  with open(path_to_basis, "rb") as file:
    basis = pickle.load(file)
  system.rom.build(
    phi=basis["phi"][rdim],
    psi=basis["psi"][rdim],
    **{k: basis[k] for k in ("mask", "xref", "xscale")}
  )

In [ ]:
data = {}
kwargs = dict(
  irange=irange,
  nb_workers=nb_workers,
  system=system,
  path=paths["data"],
  tlim=tlim
)
for (rom, path_to_basis) in paths["roms"].items():
  data[rom] = {}
  for rdim in rdims:
    build_rom(system, path_to_basis, rdim)
    data[rom][rdim] = evaluate_parallel(**kwargs)

In [ ]:
stats = {}
for rom in paths["roms"].keys():
  stats[rom] = {}
  for rdim in rdims:
    rdata = np.vstack([z for z in data[rom][rdim] if (z is not None)])
    rmean = np.mean(rdata, axis=0)
    i = np.flip(np.argsort(rmean))
    stats[rom][rdim] = {
      "size": len(rdata),
      "mean": rmean[i],
      "std": np.std(rdata, axis=0)[i],
      "sem": sp.stats.sem(rdata, axis=0)[i]
    }

Plotting

In [ ]:
def plot_zmean(
  x,
  y,
  labels=[r"$i$", r"$z_i$"],
  scales=["linear", "log"],
  legend_loc="best",
  figname=None,
  save=False,
  show=False
):
  # Initialize figures
  fig = plt.figure()
  ax = fig.add_subplot()
  # x axis
  ax.set_xlabel(labels[0])
  ax.set_xscale(scales[0])
  ax.set_xticks(x, labels=x.astype(str))
  # y axis
  ax.set_ylabel(labels[1])
  ax.set_yscale(scales[1])
  # Plotting
  for (k, yk) in y.items():
    ax.errorbar(
      x=x,
      y=yk["mean"],
      # yerr=yk["std"],
      marker='^',
      markersize=15,
      linestyle='none',
      lw=1.0,
      label=k
    )
  ax.legend(loc=legend_loc, fancybox=True, framealpha=0.9)
  # Tight layout
  plt.tight_layout()
  if save:
    plt.savefig(figname)
  if show:
    plt.show()
  plt.close()

In [ ]:
for rdim in rdims:
  plot_zmean(
    x=np.arange(rdim)+1,
    y={k: stat[rdim] for (k, stat) in stats.items()},
    labels=[r"$i$", r"$z_i$"],
    scales=["linear", "log"],
    legend_loc="best",
    figname=paths["out"] + f"/zmean_r{rdim}.png",
    save=True,
    show=False
  )